In [ ]:
NAME = '' 

In [ ]:
# 필요한 라이브러리 및 모듈 불러오기
import jchm                               # 사용자 정의 로봇 카메라 모듈 (예: JAJUCHA용)
import torch                              # PyTorch 라이브러리
import torchvision.transforms as transforms  # 이미지 전처리를 위한 torchvision 도구
from PIL import Image as Img              # PIL(Pillow) 이미지 처리 라이브러리
import cv2                                # OpenCV 이미지 처리 라이브러리
import numpy as np                        # 배열 처리

# 이미지 전처리 정의
# - 입력 이미지를 224x224로 리사이징
# - 이미지를 텐서(tensor) 형식으로 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # 이미지 크기 조정
    transforms.ToTensor(),               # PIL 이미지를 Tensor로 변환 (0~1 범위로 정규화됨)
])

# 사용할 디바이스 설정 (GPU 사용)
device = torch.device('cuda')

# 저장된 TorchScript 모델 불러오기
model = torch.jit.load('checkpoints/train/weights/' + NAME)  # NAME은 파일명 (예: 'resnet_epoch_10.pt')
model.to(device)                 # 모델을 GPU로 이동
model.eval()                     # 모델을 평가 모드로 전환 (dropout, batchnorm 비활성화)

# 실시간 예측 루프
while True:
    image = jchm.camera.get_image('center')  # 'center' 카메라에서 이미지 한 장 캡처

    # OpenCV 이미지(BGR)를 PIL 이미지(RGB)로 변환
    image_pil = Img.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) 

    # 이미지 전처리 (리사이즈 → 텐서로 변환)
    input_tensor = transform(image_pil)

    # 배치 차원 추가 (모델 입력: [batch_size, channel, height, width])
    input_tensor = input_tensor.unsqueeze(0).to(device)

    # 모델 예측 수행
    output = model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()  # 예측된 클래스 인덱스 

    # 예측된 클래스 이름을 이미지 오른쪽 상단에 표시
    # - OpenCV는 BGR 기반, 한글을 사용할 경우 한글 폰트 경로 필요
    annotated_image = image.copy()
    cv2.putText(
        annotated_image,            # 표시할 이미지
        str(predicted_class),                 # 텍스트 내용 (예측된 클래스 이름)
        (10, 30),                   # 텍스트 위치 (왼쪽 상단)
        cv2.FONT_HERSHEY_SIMPLEX,   # 글꼴
        1.0,                        # 글꼴 크기
        (0, 255, 0),                # 글자 색 (녹색)
        2                           # 두께
    )

    # 주석이 추가된 이미지를 화면에 출력
    jchm.camera.show_image(annotated_image)

    # 터미널에도 예측 결과 출력 (한 줄에 계속 갱신)
    print(f"\r예측 클래스: {predicted_class} | 출력값: {output.cpu().detach().numpy()}", end="")
